In [ ]:
! pip install keras==2.2.5

## BatchNormalization

layer의 입력으로 들어가는 데이터가 정규분포를 따른다고 layer를 통과한 출력 데이터의 분포가 정규분포를 따른다고 기대하기 어려움

BatchNormalization은 훈련하는 동안 평균과 분산이 변하더라도 이에 적응하여 데이터를 정규화 함

 - 훈련 과정에서 사용된 배치 데이터의 평균과 분산에 대한 지수 이동 평균을 내부적으로 유지 (최근 값에 가중치를 더 부여)
 - 잔차 연결과 유사하게 그래디언트의 전파를 도와줌 (보다 깊은 네트워크 구성 가능)

입력에 비해 활성화 함수의 출력이 너무 작거나 커지면 변화율이 급격히 작아져(시그모이드 함수 고려) 역전파로 전달되는 그래디언트도 크게 감소하게 됨

 - BatchNormalization은 입력과 출력의 분포가 유지되도록 하여 그래디언트가 보다 잘 전파될 수 있도록 함

### Motivation

Batch Normalization은 기본적으로 Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나

입력 x는 W, b 및 아래의 모든 계층의 매개 변수에 의해 영향을 받기 때문에 훈련 중에 이러한 매개 변수를 변경하면 x의 많은 차원이 비선형성의 포화 영역으로 이동하고 수렴 속도가 느려질 수 있음

Activation function으로 sigmoid, tanh 과 같은 비선형 포화함수를 사용하면, 입력의 값이 작은 구간을 제외하면 미분값이 0에 가까워지기 때문에 back-prop을 통한 학습이 어렵거나 느려짐.

네트워크가 학습 할 때 비선형 입력의 분포가 더 안정적으로 유지되도록 할 수 있다면 optimizer가 포화 상태에 갇힐 가능성이 줄어들고 학습이 가속화됨

Internal Covariance Shift : 네트워크의 각 층이나 활성화층 마다 입력의 분포가 달라지는 현상 (학습 불안정의 원인이 됨)

 - 평균 0, 표준편차 1로 정규화 하는 방식 대신 BatchNormalization이 대안으로 제시됨 (단순히 평균, 표준편차를 0, 1로 고정시키는 것은 오히려 Activation function의 비선형성을 없앨 수 있음)

 - 학습 데이터 전체에 대해 평균, 분산을 구하는 것이 아니라 mini batch 단위로 접근하여 계산





## Inference & Training

학습 시에는 현재 입력으로 들어온 mini batch에서 평균과 표준편차를 계산

테스트 데이터로 inference를 수행할 경우 학습 시 현재까지 본 입력들의 이동평균을 계산하여 이 저장해놓은 값으로 normalize (gamma, beta로 scale/shift 하는 것은 동일하게 적용)

학습 시에는 mini batch의 평균, 분산으로 normalize하고 inference시에는 계산해놓은 이동 평균의 값으로 normalize


![Fig7-3-3-9](https://user-images.githubusercontent.com/44194558/142820113-507d22c4-53b7-4ff2-925a-2f71ecbe2306.PNG)

![Fig7-3-3-11](https://user-images.githubusercontent.com/44194558/142820152-c2d3c4c0-9f24-4440-9bfa-13959b272b5e.PNG)

## Summary

Proposed a new mechanism, Batch Normalization, that takes a step towards reducing internal covariate shift, and in doing so dramatically accelerates the training of deep neural nets.

It accomplishes this via a normalization step that fixes the means and variances of layer inputs

Batch Normalization also has a beneficial effect on the gradient flow through the network, by reducing the dependence of gradients on the scale of the parameters or of their initial values.

This allows us to use much higher learning rates without the risk of divergence.
Batch normalization regularizes the model and reduces the need for Dropout (Srivastava et al., 2014).

Batch Normalization makes it possible to use saturating nonlinearities by preventing the network from getting stuck in the saturated modes.

In [ ]:
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

dense_model.add(layers.Dense(32, activation='relu'))
dense_model.add(layers.BatchNormalization())

## Depthwise separable convolution

입력 채널별로 따로 따로 공간 방향의 합성곱을 수행(depthwise), 그 다음에 pointwise 합성곱(1x1 합성곱)을 통해 출력 채널을 합침

![Fig7-3-9](https://user-images.githubusercontent.com/44194558/142820614-7d081d82-3144-42b1-92c4-44d48a3608a3.PNG)

In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3,
                                 activation='relu',
                                 input_shape=(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

## Hyperparameter 최적화

![Fig7-3-11](https://user-images.githubusercontent.com/44194558/142840934-88a8c056-07d8-4b5f-adeb-7b922e5ca760.PNG)

## 모델 앙상블

분류기의 다양성이 핵심

독립적으로 훈련된 서로 다른 모델들이 각기 다른 장점을 가지고 있다는 장점을 가지고 있다는 가정

![Fig7-3-14](https://user-images.githubusercontent.com/44194558/142841164-b4719b75-e697-49b0-9ca4-039929cb2c4b.PNG)